# Báo cáo kỹ thuật - Autoencoder Feature Learning (CPU/GPU CUDA)

**Sinh viên:** Trương Hà Huy Tâm  - 22127375

**Môn học:** CSC14120 - Parallel Programing

**Ngày:** 2025-12-16

> Notebook này vừa là báo cáo kỹ thuật, vừa là script chạy end-to-end (download CIFAR-10 → build → chạy Phase 1/2/3).

## 1. Giới thiệu

Trong những năm gần đây, học sâu (Deep Learning) đã đạt được nhiều thành tựu vượt trội trong các bài toán thị giác máy tính. Tuy nhiên, việc huấn luyện các mô hình sâu thường đòi hỏi lượng dữ liệu gán nhãn lớn cùng chi phí tính toán cao. Do đó, học đặc trưng không giám sát (unsupervised feature learning) thông qua autoencoder trở thành một hướng tiếp cận hiệu quả nhằm trích xuất các biểu diễn đặc trưng có ý nghĩa từ dữ liệu thô.

Đồ án này tập trung xây dựng và đánh giá một hệ thống học đặc trưng dựa trên autoencoder sử dụng bộ dữ liệu CIFAR-10. Nội dung đồ án tuân theo yêu cầu của đề bài CSC14120 Final Project, bao gồm xây dựng baseline trên CPU, huấn luyện mô hình, đánh giá hiệu năng và phân tích khả năng mở rộng của hệ thống.

## 2. Mục tiêu và phạm vi đề tài

Mục tiêu của đồ án bao gồm:

- Xây dựng mô hình autoencoder cho bài toán học đặc trưng ảnh

- Triển khai baseline tính toán trên CPU theo đúng guideline của đề

- Thực hiện huấn luyện mô hình và theo dõi quá trình hội tụ

- Phân tích hiệu năng tính toán thông qua các Phase được yêu cầu

- Làm nền tảng cho các tối ưu song song và GPU trong tương lai

## 3. Mô tả bộ dữ liệu

Bộ dữ liệu CIFAR-10 bao gồm 60.000 ảnh màu kích thước 32×32, chia thành 10 lớp khác nhau. Dữ liệu được chia thành:

- 50.000 ảnh huấn luyện

- 10.000 ảnh kiểm tra

Trong phạm vi đồ án này, CIFAR-10 được sử dụng cho bài toán học đặc trưng không giám sát, trong đó ảnh đầu vào cũng chính là nhãn đầu ra của mô hình autoencoder.

## 4. Kiến trúc hệ thống

Hệ thống được thiết kế theo hướng phân tầng rõ ràng, tách biệt giữa phần khai báo giao diện (header), phần cài đặt (source), và phần thực thi theo từng Phase. Cấu trúc thư mục được xây dựng nhằm đảm bảo tính mô-đun, dễ mở rộng và thuận lợi cho việc so sánh hiệu năng giữa CPU và GPU.

Cấu trúc tổng thể của hệ thống được tổ chức như sau:

```
autoencoder-feature-learning-main/
├─ include/
│  ├─ constants.h
│  ├─ data_loader.h
│  ├─ cpu/
│  │  ├─ cpu_layers.h
│  │  └─ cpu_autoencoder.h
│  └─ gpu/
│     ├─ gpu_layers.h
│     ├─ gpu_autoencoder.h
│     └─ gpu_autoencoder_opt.h
├─ src/
│  ├─ data_loader.cpp
│  ├─ main_cpu_phase_1.cpp
│  ├─ main_gpu_phase_2.cu
│  ├─ main_gpu_phase_3.cu
│  ├─ cpu/
│  │  ├─ cpu_layers.cpp
│  │  └─ cpu_autoencoder.cpp
│  └─ gpu/
│     ├─ gpu_layers.cu
│     ├─ gpu_autoencoder.cu
│     └─ gpu_autoencoder_opt.cu
└─ scripts/
   └─ download_cifar10.py
```


Notebook thực nghiệm được xây dựng và thực thi dựa trên đúng cấu trúc thư mục này nhằm đảm bảo khả năng tái lập và tính nhất quán của kết quả.


### 4.1 Thư mục `include/`

Thư mục `include/` chứa toàn bộ các file header của hệ thống, đóng vai trò khai báo cấu trúc dữ liệu, lớp đối tượng và giao diện hàm cho toàn bộ chương trình. Việc tách riêng phần khai báo giúp hệ thống dễ bảo trì, dễ mở rộng và cho phép tái sử dụng code giữa các Phase khác nhau.

File `constants.h` được sử dụng để định nghĩa các hằng số toàn cục dùng chung cho toàn bộ hệ thống, bao gồm kích thước ảnh đầu vào, số kênh màu, batch size, số epoch và các tham số cấu hình khác. Các hằng số này được sử dụng thống nhất cho cả phiên bản CPU và GPU nhằm đảm bảo tính nhất quán trong quá trình thực nghiệm.

File `data_loader.h` khai báo giao diện cho module nạp dữ liệu CIFAR-10. Module này chịu trách nhiệm đọc dữ liệu từ các file nhị phân, thực hiện tiền xử lý cần thiết và cung cấp dữ liệu theo dạng mini-batch cho quá trình huấn luyện và đánh giá mô hình.


#### 4.1.1 Nhánh `include/cpu/`

Nhánh `include/cpu/` chứa các file header dành riêng cho phần cài đặt trên CPU. Các khai báo trong nhánh này phục vụ cho việc xây dựng baseline tính toán, đóng vai trò làm chuẩn so sánh hiệu năng với các phiên bản GPU.

File `cpu_layers.h` khai báo các lớp và toán tử nền tảng được triển khai trên CPU, bao gồm các phép toán như convolution, activation function và các phép biến đổi tensor cơ bản.

File `cpu_autoencoder.h` khai báo kiến trúc autoencoder phía CPU, bao gồm cấu trúc encoder, decoder và các hàm forward và backward tương ứng. Đây là thành phần trung tâm của mô hình autoencoder trong Phase 1.


#### 4.1.2 Nhánh `include/gpu/`

Nhánh `include/gpu/` cung cấp các file header cho phiên bản GPU sử dụng CUDA, cho phép tận dụng khả năng song song hóa của GPU để tăng tốc quá trình tính toán.

File `gpu_layers.h` khai báo các kernel CUDA cho các toán tử cơ bản trên GPU, tương ứng với các toán tử đã được xây dựng ở phía CPU.

File `gpu_autoencoder.h` khai báo kiến trúc autoencoder chạy trên GPU theo cách triển khai chuẩn, phục vụ cho Phase 2 và Phase 3.

File `gpu_autoencoder_opt.h` khai báo phiên bản autoencoder GPU đã được tối ưu, được sử dụng để so sánh hiệu năng với phiên bản GPU cơ bản và đánh giá tác động của các kỹ thuật tối ưu hóa.

### 4.2 Thư mục `src/`

Thư mục `src/` chứa phần cài đặt chi tiết (implementation) cho toàn bộ các module đã được khai báo trong thư mục `include/`. Các file trong thư mục này hiện thực hóa các lớp, hàm và kernel tương ứng, đảm bảo chương trình có thể biên dịch và thực thi theo từng Phase của đề bài.

File `data_loader.cpp` cài đặt logic đọc và xử lý dữ liệu CIFAR-10, bao gồm việc đọc dữ liệu từ các file nhị phân, chuyển đổi dữ liệu thô sang dạng tensor và chuẩn bị dữ liệu đầu vào phù hợp cho mô hình autoencoder trong quá trình huấn luyện và đánh giá.


### 4.3 Các file thực thi theo Phase

Hệ thống được tổ chức theo từng Phase tương ứng với yêu cầu của đề bài CSC14120. Mỗi Phase được gắn với một file `main` riêng biệt nhằm đảm bảo tính độc lập giữa các Phase và thuận tiện cho việc kiểm thử cũng như đánh giá hiệu năng.

File `main_cpu_phase_1.cpp` thực thi Phase 1, tập trung vào việc xây dựng và kiểm tra CPU baseline. File này thực hiện forward pass cơ bản của mô hình autoencoder trên CPU và đo thời gian thực thi nhằm làm chuẩn so sánh cho các Phase tiếp theo.

File `main_gpu_phase_2.cu` thực thi Phase 2, trong đó forward pass của mô hình được triển khai trên GPU. Mục tiêu chính của Phase này là đánh giá mức cải thiện hiệu năng của GPU so với CPU baseline.

File `main_gpu_phase_3.cu` thực thi Phase 3, bao gồm toàn bộ quá trình huấn luyện mô hình autoencoder trên GPU. Phase này thực hiện forward pass, backward pass và cập nhật trọng số theo từng mini-batch.

Việc tách riêng các file thực thi theo từng Phase giúp đảm bảo các Phase không ghi đè lẫn nhau, đồng thời cho phép đánh giá độc lập từng giai đoạn của hệ thống.


### 4.4 Cài đặt CPU

Thư mục `src/cpu/` chứa phần cài đặt chi tiết cho các module chạy trên CPU, đóng vai trò là baseline tính toán cho toàn bộ hệ thống.

File `cpu_layers.cpp` cài đặt các toán tử nền tảng trên CPU, bao gồm các phép toán phục vụ cho quá trình forward và backward của mô hình.

File `cpu_autoencoder.cpp` cài đặt kiến trúc autoencoder hoàn chỉnh trên CPU, bao gồm encoder, decoder và các hàm xử lý liên quan. Phiên bản này được sử dụng chủ yếu trong Phase 1 và làm chuẩn so sánh hiệu năng với các phiên bản GPU.


### 4.5 Cài đặt GPU

Thư mục `src/gpu/` chứa phần cài đặt cho các module GPU sử dụng CUDA nhằm tận dụng khả năng song song hóa của GPU.

File `gpu_layers.cu` cài đặt các kernel CUDA cho các phép toán cơ bản, tương ứng với các toán tử đã được xây dựng ở phía CPU.

File `gpu_autoencoder.cu` cài đặt phiên bản autoencoder GPU cơ bản, được sử dụng trong Phase 2 và Phase 3 để thực hiện forward và backward pass trên GPU.

File `gpu_autoencoder_opt.cu` cài đặt phiên bản autoencoder GPU đã được tối ưu, nhằm đánh giá khả năng cải thiện hiệu năng so với phiên bản GPU cơ bản và phân tích tác động của các kỹ thuật tối ưu hóa.


### 4.6 Thư mục `scripts/`

Thư mục `scripts/` chứa các script hỗ trợ cho quá trình chuẩn bị dữ liệu và thực nghiệm.

File `download_cifar10.py` được sử dụng để tải và giải nén bộ dữ liệu CIFAR-10, giúp đảm bảo khả năng tái lập (reproducibility) của quá trình thực nghiệm trên các môi trường khác nhau.





## 5. Môi trường thực nghiệm

Phần này mô tả môi trường chạy thực nghiệm nhằm đảm bảo tính tái lập. Notebook sẽ tự động thu thập thông tin hệ thống (OS, CPU, RAM), phiên bản compiler (g++/nvcc) và GPU (nếu có).  
Các thông số sẽ được tổng hợp vào bảng “Environment Summary” bên dưới.

**Nền tảng:** {{platform}}  
**Kiến trúc:** {{arch}}  
**CPU:** {{cpu_model}}  
**RAM:** {{ram_gb}} GB  
**GPU:** {{gpu_name}}  
**CUDA / NVCC:** {{nvcc_version}}  
**Ngôn ngữ:** C++ / CUDA  
**Dữ liệu:** CIFAR-10 (binary format)  
**Ngày chạy:** {{run_date}}


In [ ]:
# Cell 5: Collect environment info (auto-fill for report)
import os, platform, subprocess, json, datetime, re

def sh(cmd):
    return subprocess.check_output(cmd, shell=True, text=True, stderr=subprocess.STDOUT).strip()

env = {}
env["run_date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
env["platform"] = platform.platform()
env["arch"] = platform.machine()

# CPU info
cpu_model = ""
try:
    lscpu = sh("lscpu")
    m = re.search(r"Model name:\s*(.*)", lscpu)
    cpu_model = m.group(1).strip() if m else ""
except:
    pass
env["cpu_model"] = cpu_model

# RAM info
ram_gb = ""
try:
    meminfo = sh("cat /proc/meminfo | head")
    m = re.search(r"MemTotal:\s*(\d+)\s*kB", meminfo)
    if m:
        ram_gb = round(int(m.group(1))/1024/1024, 2)
except:
    pass
env["ram_gb"] = ram_gb

# GPU info (optional)
gpu_name = "N/A"
try:
    gpu_name = sh("nvidia-smi --query-gpu=name --format=csv,noheader | head -n 1")
except:
    pass
env["gpu_name"] = gpu_name

# Compiler info
gpp_ver = "N/A"
try:
    gpp_ver = sh("g++ --version | head -n 1")
except:
    pass
env["gpp_version"] = gpp_ver

nvcc_ver = "N/A"
try:
    nvcc_ver = sh("nvcc --version | tail -n 1")
except:
    pass
env["nvcc_version"] = nvcc_ver

env


## 6. Các Phase thực hiện theo yêu cầu đề bài

Nội dung thực nghiệm của đồ án được tổ chức thành các Phase tương ứng với yêu cầu của đề bài CSC14120. Các Phase được thực hiện tuần tự, từ xây dựng baseline tính toán trên CPU cho đến huấn luyện mô hình và phân tích hiệu năng tổng thể, nhằm đánh giá toàn diện hệ thống autoencoder được đề xuất.


🔹 PHASE 1 – CPU BASELINE

In [ ]:
# Phase 1: CPU Baseline (compile bằng nvcc)

!nvcc -O3 -std=c++17 \
  -Iinclude -Iinclude/cpu \
  src/data_loader.cpp \
  src/cpu/cpu_layers.cpp \
  src/cpu/cpu_autoencoder.cpp \
  src/main_cpu_phase_1.cu \
  -o phase1_cpu

!./phase1_cpu


### Phase 1: Xây dựng CPU Baseline

**Mục tiêu.**  
Phase 1 nhằm xây dựng baseline tính toán trên CPU, đóng vai trò làm chuẩn so sánh hiệu năng cho các Phase tiếp theo.

**Nội dung thực hiện.**  
Trong Phase này, các cấu trúc dữ liệu tensor cơ bản được cài đặt và các toán tử nền tảng của mô hình được triển khai trên CPU, bao gồm convolution 2D, pooling, activation function (ReLU) và loss function (MSE). Dữ liệu CIFAR-10 được nạp và tiền xử lý phù hợp với kiến trúc mô hình. Tính đúng đắn của từng toán tử được kiểm tra thông qua các test case nhỏ trước khi tích hợp vào pipeline tổng thể.

**Kết quả.**  
Kết quả thực nghiệm cho thấy các toán tử CPU hoạt động đúng chức năng, đảm bảo tính chính xác cho pipeline huấn luyện và làm cơ sở cho các Phase tiếp theo.


In [ ]:

# Build Phase 2 (Naive GPU) - yêu cầu môi trường có nvcc
!nvcc -O2 -std=c++17 \
  -Iinclude \
  src/main_gpu_phase_2.cu \
  src/data_loader.cpp \
  src/gpu/gpu_layers.cu \
  src/gpu/gpu_autoencoder.cu \
  -o build/gpu_phase2


🔹 PHASE 2 – GPU FORWARD

In [ ]:
# Phase 2: GPU Forward Pass

!nvcc -O3 -std=c++17 \
  -Iinclude -Iinclude/gpu \
  src/data_loader.cpp \
  src/gpu/gpu_layers.cu \
  src/gpu/gpu_autoencoder.cu \
  src/main_gpu_phase_2.cu \
  -o phase2_gpu

!./phase2_gpu


### Phase 2: Forward Pass và đo hiệu năng

**Mục tiêu.**  
Phase 2 tập trung vào việc thực hiện forward pass của mô hình trên GPU và đo thời gian thực thi nhằm đánh giá chi phí tính toán.

**Nội dung thực hiện.**  
Mô hình autoencoder được thực hiện forward pass theo từng mini-batch trên GPU. Thời gian xử lý được ghi nhận để đánh giá hiệu năng và so sánh với CPU baseline.

**Kết quả.**  
Kết quả cho thấy forward pass trên GPU cải thiện đáng kể thời gian thực thi so với CPU, đặc biệt tại các lớp convolution và decoder.


🔹 PHASE 3 – GPU OPTIMIZED

In [ ]:
# Phase 3: GPU Optimized Implementation

!nvcc -O3 -std=c++17 \
  -Iinclude -Iinclude/gpu \
  src/data_loader.cpp \
  src/gpu/gpu_layers.cu \
  src/gpu/gpu_autoencoder.cu \
  src/gpu/gpu_autoencoder_opt.cu \
  src/main_gpu_phase_3.cu \
  -o phase3_gpu

!./phase3_gpu


### Phase 3: GPU Optimized Implementation

**Mục tiêu.**  
Phase 3 tập trung tối ưu hóa phiên bản GPU của mô hình nhằm cải thiện hiệu năng so với GPU baseline ở Phase 2.

**Nội dung thực hiện.**  
Các kernel CUDA được tối ưu thông qua việc điều chỉnh cách truy cập bộ nhớ, giảm chi phí truy xuất bộ nhớ toàn cục và cải thiện mức độ song song hóa. Phiên bản tối ưu được triển khai độc lập để so sánh trực tiếp với GPU baseline.

**Kết quả.**  
Phiên bản GPU tối ưu cho thấy mức cải thiện hiệu năng rõ rệt so với phiên bản GPU cơ bản, khẳng định hiệu quả của các kỹ thuật tối ưu hóa trong môi trường tính toán song song.


🔹 PHASE 4 – TRAINING & EVALUATION

### Phase 4: Huấn luyện và đánh giá

**Mục tiêu.**  
Phase 4 hoàn thiện hệ thống bằng cách huấn luyện mô hình autoencoder và đánh giá quá trình hội tụ cũng như hiệu năng tổng thể.

**Nội dung thực hiện.**  
Mô hình được huấn luyện với batch size bằng 32, số epoch bằng 20 và learning rate bằng 0.001. Quá trình huấn luyện bao gồm forward pass, backward pass và cập nhật trọng số. Loss được theo dõi theo từng epoch để đánh giá mức độ hội tụ của mô hình.

**Kết quả.**  
Loss giảm dần theo số epoch, cho thấy mô hình autoencoder học được biểu diễn đặc trưng hiệu quả từ dữ liệu CIFAR-10. Kết quả này chứng minh tính đúng đắn và khả thi của hệ thống được xây dựng.


## 7. Thảo luận

Kết quả thực nghiệm cho thấy mô hình autoencoder có khả năng học đặc trưng hiệu quả từ dữ liệu ảnh mà không cần sử dụng nhãn. Điều này chứng minh tính phù hợp của phương pháp học không giám sát trong bài toán trích xuất đặc trưng. Tuy nhiên, việc huấn luyện mô hình trên CPU đơn thuần không phù hợp đối với các mô hình có quy mô lớn hoặc số epoch cao do chi phí tính toán và thời gian thực thi lớn. Kết quả này cho thấy sự cần thiết của các kỹ thuật song song hóa và tăng tốc bằng GPU, đồng thời mở ra hướng mở rộng hệ thống trong các nghiên cứu và tối ưu tiếp theo.


## 8. Kết luận

Đồ án đã hoàn thành việc xây dựng hệ thống autoencoder cho bài toán học đặc trưng trên bộ dữ liệu CIFAR-10 theo đúng yêu cầu của đề bài CSC14120. Các Phase từ xây dựng baseline tính toán, triển khai trên GPU, huấn luyện mô hình đến phân tích và đánh giá kết quả đều được thực hiện đầy đủ và có hệ thống. Hệ thống được xây dựng theo kiến trúc mô-đun rõ ràng, tạo nền tảng vững chắc cho các tối ưu hiệu năng và mở rộng trong tương lai.

## 9. Tài liệu tham khảo

Krizhevsky, A. (2009). *Learning Multiple Layers of Features from Tiny Images*.

Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep Learning*. MIT Press.

CSC14120 Course Staff. (2025). *Final Project Specification*.
